In [1]:
import pandas as pd

# Part 1

We are going to use Pandas built in html parser.
Due to SSL issues I had to download the file to my local machine. Not a big deal.

In [2]:
page = pd.read_html('List of postal codes.htm',flavor ='bs4')
page

[            0                 1  \
 0    Postcode           Borough   
 1         M1A      Not assigned   
 2         M2A      Not assigned   
 3         M3A        North York   
 4         M4A        North York   
 5         M5A  Downtown Toronto   
 6         M5A  Downtown Toronto   
 7         M6A        North York   
 8         M6A        North York   
 9         M7A      Queen's Park   
 10        M8A      Not assigned   
 11        M9A         Etobicoke   
 12        M1B       Scarborough   
 13        M1B       Scarborough   
 14        M2B      Not assigned   
 15        M3B        North York   
 16        M4B         East York   
 17        M4B         East York   
 18        M5B  Downtown Toronto   
 19        M5B  Downtown Toronto   
 20        M6B        North York   
 21        M7B      Not assigned   
 22        M8B      Not assigned   
 23        M9B         Etobicoke   
 24        M9B         Etobicoke   
 25        M9B         Etobicoke   
 26        M9B         Etobi

It seems we only need the first table.

In [3]:
df = page[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Let's use the first row as the header.

In [4]:
df.rename(columns=df.iloc[0], inplace = True) # Use first row as headers
df.drop(df.index[0],inplace = True) # Drop the 1st row
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Now lets:
    1. Drop the postcodes without borough assigned.
    2. Aggregate neighborhoods in the same postal code.
    3. Assign borough as a neighbrhood name where there is none.

In [5]:
df = df[df.Borough != 'Not assigned'] # Drop the postcodes without borough assigned.
df = df.groupby('Postcode').agg(lambda x:', '.join(set(x))) # More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough'] # Assign borou

In [6]:
df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"West Hill, Morningside, Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


That's all folks!

# Part 2

Well, Google API is garbage, so I am going to use the provided .csv

In [7]:
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Going to merge both dataframes on Postal Codes.

In [8]:
df2 = pd.merge(df,geo,left_on=['Postcode'], right_on = ['Postal Code'], how = 'left')

In [9]:
df2.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"West Hill, Morningside, Guildwood",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


And Finally, some cleaning:

In [10]:
df2.rename(columns={'Postal Code':'PostalCode'},inplace = True)
neighborhoods = df2[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]

In [11]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

Since I lack imagination, I am going to do a similar analysis to the one done in the lab.
Lets start with filtering only boroughs inside Toronto:

In [12]:
toronto_neigh = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]
toronto_neigh['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

Let's get the coordinates for Toronto:

In [13]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


And create a map to see we are in a right direction

In [14]:
import folium
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=12)
for lat, lng, label in zip(toronto_neigh['Latitude'], toronto_neigh['Longitude'], toronto_neigh['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

Let's initialize our credentials and import the needed libs:

In [15]:
import requests
CLIENT_ID = 'A4B3N0TNM1SYNDB5U1ZJCIKTEQRWNZYPO0YQLJEDO1E1PAGT'
CLIENT_SECRET = 'FGLDG2V50BTNMPANGCH5T3ESSITZHXTJ1TFED0PVU3WIT54T'
VERSION = '20180605'

And borrow the functions from the lab (With small modifications):

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's get the info from foursquares:

In [17]:
toronto_venues = getNearbyVenues(names=toronto_neigh['Neighbourhood'],
                                   latitudes=toronto_neigh['Latitude'],
                                   longitudes=toronto_neigh['Longitude']
                                  )

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
King, Adelaide, Richmond
Union Station, Harbourfront East, Toronto Islands
Toronto Dominion Centre, Design Exchange
Victoria Hotel, Commerce Court
Roselawn
Forest Hill West, Forest Hill North
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, Railway Lands, CN Tower, Island airport, South Niagara, Harbourfront West, King and Spadina
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Parkdale Village, Exhibition Place, Brockton
High Park, The 

Let's see what we got:

In [18]:
toronto_venues.shape

(1700, 7)

In [19]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [20]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Quay, Railway Lands, CN Tower, Island airport, South Niagara, Harbourfront West, King and Spadina",15,15,15,15,15,15
Berczy Park,57,57,57,57,57,57
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
Davisville,32,32,32,32,32,32


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


Seems like the data for Toronto is smaller then for NY, but it should be enough for our model.

Let's find the average frequency of each venue category per neighborhood.
Starting by encoding the categories and calculating the mean per neighbourhood:

In [22]:
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") #Encode Categories
onehot['Neighbourhood']  = toronto_venues['Neighbourhood']
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns] #Move Neighbourhood to first column
toronto_grouped = onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Bathurst Quay, Railway Lands, CN Tower, Island...",0.0,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0,0.000000
2,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.058824
3,"Cabbagetown, St. James Town",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022727,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.0,0.0,0.0,0.0,0.011364,0.0,0.0,0.011364,0.0,0.011364


Finally, let's create a DF with a neighbourhood and it's top 10 venues:

In [23]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Quay, Railway Lands, CN Tower, Island...",Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery
2,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
3,"Cabbagetown, St. James Town",Coffee Shop,Pub,Pizza Place,Restaurant,Café,Market,Italian Restaurant,Bakery,Playground,Liquor Store
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant,Middle Eastern Restaurant,Salad Place,Restaurant,Bubble Tea Shop,Bar


Let's run k-means algorith to create our clusters

In [24]:
from sklearn.cluster import KMeans
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

And tidy up the data:

In [25]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_neigh
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Furniture / Home Store,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4,Park,Sandwich Place,Pet Store,Ice Cream Shop,Burger Joint,Liquor Store,Light Rail Station,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
43,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Latin American Restaurant,Coworking Space
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Let's put it on the map:

In [26]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Interesting Results. Let's check each cluster now:

In [27]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [28]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,1,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [29]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [30]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
48,Central Toronto,3,Playground,Tennis Court,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
50,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [31]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Furniture / Home Store,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,East Toronto,4,Park,Sandwich Place,Pet Store,Ice Cream Shop,Burger Joint,Liquor Store,Light Rail Station,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
43,East Toronto,4,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Latin American Restaurant,Coworking Space
45,Central Toronto,4,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Clothing Store,Hotel,Discount Store,Dog Run,Doner Restaurant
46,Central Toronto,4,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Pet Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop,Diner
47,Central Toronto,4,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gourmet Shop,Dance Studio,Seafood Restaurant
49,Central Toronto,4,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Convenience Store,Light Rail Station
51,Downtown Toronto,4,Coffee Shop,Pub,Pizza Place,Restaurant,Café,Market,Italian Restaurant,Bakery,Playground,Liquor Store
52,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Nightclub,Bubble Tea Shop,Men's Store,Café,Mediterranean Restaurant
53,Downtown Toronto,4,Coffee Shop,Park,Pub,Café,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Brewery,Italian Restaurant
